# 0 Import modules

In [8]:
import obspy
from obspy.io.sac.sactrace import SACTrace
from obspy.io.sac.util import get_sac_reftime
import os
import glob
import shutil
from obspy.clients.iris import Client
from obspy.taup import TauPyModel
import pandas as pd
import re

client = Client()
model = TauPyModel(model= 'hk.npz')

# 1 Resample to 40Hz and bandpass filtered

In [11]:
events = pd.read_csv("events.txt", header = None, delim_whitespace=True)


# Function to remove non-numerical symbols from a string
def remove_non_numerical(s):
    return re.sub('[^0-9]', '', s)

# Apply the function to the 'col1' column of the DataFrame
events[3] = events[3].apply(remove_non_numerical)
events[0][1]

39901391

In [53]:
df_verified = pd.read_csv("verified/verified.txt", names=["evt_id","sta", "Tp", 
                                                   "p_len","TsPmP", "sPmP_len",
                                                   "TsPn", "sPn_len", "quality", "evt_num"], header = None)
#df_verified.loc[df_verified["evt_num"] == 20220112021208390].index[0]
df_verified['evt_num'] = df_verified['evt_num'].astype(str)

# print(df_verified.dtypes)
# print(type(df_verified["evt_num"][0]))
#df_verified

In [24]:
data_dir = "Good_SAC_files"
for ev in os.listdir(data_dir):
    ev= int(ev)
    print(int(ev))
    print(type(ev))
    break

20220211141550450
<class 'int'>


In [54]:
df_verified

,evt_id,sta,Tp,p_len,TsPmP,sPmP_len,TsPn,sPn_len,quality,evt_num
0,39909423,CI.HOL.,15.558,0.3,18.173,0.3,nan,0.4,a,20220112002458770
1,39909511,AZ.LVA2.,3.723,0.2,11.662,0.3,nan,0.4,b,20220112021208390
2,39909511,AZ.SMER.,12.748,0.2,16.068,0.3,nan,0.4,b,20220112021208390
3,39909511,BC.RMX.,16.035,0.3,19.083,0.3,18.483,0.3,a,20220112021208390
4,39909663,CI.EML.,18.840,0.4,23.763,0.4,22.793,0.4,a,20220112090603170
...,...,...,...,...,...,...,...,...,...,...
96,40180784,CI.IKP.,13.600,0.4,15.700,0.4,nan,0.3,c,20220209121834630
97,40182952,CI.SRT.,4.950,0.4,11.700,0.4,nan,0.3,c,20220211121842170
98,40183048,CI.SRT.,4.920,0.4,12.000,0.4,nan,0.3,b,20220211141550450
99,40183056,CI.CCA.,15.000,0.4,18.200,0.4,nan,0.3,a,20220211142113660


In [60]:
# I have resampled the waveforms before, 
# I just need to add the nevid and sP arrival into the SAC file

# The same is true fo Filter and AlignP

#write evt_id into SAC and write sPmP arrival time into SAC file
data_dir = "Good_SAC_files"
data_dest = "SAC_for_readin"
for ev in os.listdir(data_dir):
    inpath = os.path.join(data_dir, ev)
    print(ev)
    
    for sac in glob.glob(f"{inpath}/*.bp.cut"):
        
        insac = sac#os.path.join(inpath, sac)
        
        
        st = obspy.read(insac)
        #print(st)
        fname = ev+'.' + st[0].stats.network + '.' + st[0].stats.station + st[0].stats.location + '.BHZ'
#             if os.path.exists(f"{data_dest}/{fname}"):
#                 continue
        evt_index = df_verified.loc[df_verified["evt_num"] == ev].index[0]
        #print(evt_index)
        st[0].stats.sac['nevid'] = str(df_verified["evt_id"][evt_index])

        # write sP arrival
        sachd = st[0].stats.sac
#             distaz = client.distaz(sachd["stla"], sachd["stlo"], sachd["evla"], sachd["evlo"])
#             gcarc = distaz['distance']
#             arrivals = model.get_travel_times(source_depth_in_km=sachd["evdp"],
#                                   distance_in_degree=gcarc, phase_list=['sPvmP'])

        TsPmP = df_verified["TsPmP"][evt_index]
        #print(TsPmP)
        st[0].stats.sac["t2"] = TsPmP + sachd["o"]
        st[0].stats.sac["kt2"] = "sPmP"
        # resample
        #st.resample(sample_rate, no_filter="False")
        #splt = sac.split('.')
        destin = os.path.join(data_dest, ev)
        if not os.path.exists(destin):
            os.makedirs(destin)
        st[0].write(f"{destin}/{fname}.SAC.bp", format='SAC')

20220211141550450
20220116091814410
20220202100105740
20220202070130500
20220211142113660
20220112021208390
20220112002458770
20220113032108850
20220116124930000
20220205074909660
20220115161902000
20220116145950590
20220201012424220
desktop.ini
20220207121028870
20220205183420810
20220122234305360
20220113082309790
20220126050555330
20220126095905010
20220209040551690
20220122010142300
20220201192630930
20220206232447510
20220117000229990
20220211121842170
20220203075810850
20220117155620910
20220123205715240
20220125172319330
20220203170822680
20220125192818690
20220129055641000
20220118073915120
20220126103830020
20220204105219770
20220128203303970
20220127095426310
20220127033401460
20220211232548160
20220128104007860
20220203151832540
20220120095230400
20220124023838010
20220123044939870
20220112090603170
20220131234528720
20220209014742470
20220113032453520
20220209121834630
20220125212130440
20220205074104630
20220118084437210
20220205234904960


# 2 Change seismograms into envelopes

# 3 label travel times for sP

# 4 Normalize envelope, standardize time step to mean 0 and variance 1

# 5 Linearly normalize dist and evdp into range [0,1]